In [ ]:
!pip install langchain-groq


In [1]:
import getpass
import os

try:
    
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

In [6]:
from langchain_groq import ChatGroq
import os

llm = ChatGroq(
    model="openai/gpt-oss-20b",   
    api_key=os.getenv("GROQ_API_KEY")  
)


In [7]:
from langchain_core.tools import tool

@tool
def angry() -> str:
    """ Call this tool if the mood is angry"""
    return "GUNS BLAZING RAMBO"


@tool
def sad() -> str:
    """ Call this tool if you want to show sad, depressed, or negative feelings"""
    return "time to forget and move on🤕"


@tool
def happy() -> str:
    """ Call this tool if you want to show happiness or positive feeling"""
    return "Miska Muska Mickeyyyyy Mouseee"

In [9]:
tools_list = [angry, sad, happy ]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

In [8]:
# we create a tool calling Agent by binding a list of tools to the llm
llm_with_tools = llm.bind_tools(tools_list)

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

# This will store all converation
chat_history = [
    SystemMessage(content="You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation")
    ]

In [16]:
chat_history.append(HumanMessage(content="World has so much crime and hatred"))

response = llm_with_tools.invoke(chat_history)
response


AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to respond. The user says: "World has so much crime and hatred". According to instructions: "You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation". The user statement expresses concern about crime and hatred. We should respond with an appropriate emotional tone. Likely sad or angry? The user expresses a negative observation. We could show empathy and sadness or anger. The instructions: "either respond the question, or show angry or sad emotions, or be happy with the situation". There\'s no question. So we either respond or show emotion. Probably show sad or angry. I\'d choose sad, as it\'s a global issue. So we call sad().\n\nWe should output a function call to sad with no arguments.', 'tool_calls': [{'id': 'fc_e38ca692-c6b1-46f7-949a-da59a89a1bef', 'function': {'arguments': '{}', 'name': 'sad'}, 'type': 'function'}]}, 

In [17]:
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='World has so much crime and hatred', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to respond. The user says: "World has so much crime and hatred". According to instructions: "You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation". The user statement expresses concern about crime and hatred. We should respond with an appropriate emotional tone. Likely sad or angry? The user expresses a negative observation. We could show empathy and sadness or anger. The instructions: "either respond the question, or show angry or sad emotions, or be happy with the situation". There\'s no question

In [25]:
chat_history.append(HumanMessage(content="Someone drank my protein shake and i am really irritated by that!"))
response = llm_with_tools.invoke(chat_history)
response

AIMessage(content='', additional_kwargs={'reasoning_content': 'The user says "Someone drank my protein shake and i am really irritated by that!" We need to respond appropriately. The user is irritated. We should show empathy and maybe suggest ways to handle. According to the developer instructions, we should respond the question, or show angry or sad emotions, or be happy with the situation. The user is irritated, so maybe sad or angry? The user is irritated, not necessarily angry. But we can show empathy. According to instructions, "Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation". The user is upset. We should respond with empathy and maybe a sad or angry tone. The user is irritated, so perhaps we can show sad or angry. We might choose sad because it\'s about irritation. But the user didn\'t explicitly ask for advice; they just expressed irritation. We can respond with a sad or angry response. Maybe a sad respon

In [19]:
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='World has so much crime and hatred', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to respond. The user says: "World has so much crime and hatred". According to instructions: "You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation". The user statement expresses concern about crime and hatred. We should respond with an appropriate emotional tone. Likely sad or angry? The user expresses a negative observation. We could show empathy and sadness or anger. The instructions: "either respond the question, or show angry or sad emotions, or be happy with the situation". There\'s no question

In [20]:
chat_history.append(tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0]))
chat_history

[SystemMessage(content='You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='World has so much crime and hatred', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to respond. The user says: "World has so much crime and hatred". According to instructions: "You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation". The user statement expresses concern about crime and hatred. We should respond with an appropriate emotional tone. Likely sad or angry? The user expresses a negative observation. We could show empathy and sadness or anger. The instructions: "either respond the question, or show angry or sad emotions, or be happy with the situation". There\'s no question

In [21]:
chat_history.append(HumanMessage(content="i met mickey mouse today"))
response = llm_with_tools.invoke(chat_history)
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='World has so much crime and hatred', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to respond. The user says: "World has so much crime and hatred". According to instructions: "You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation". The user statement expresses concern about crime and hatred. We should respond with an appropriate emotional tone. Likely sad or angry? The user expresses a negative observation. We could show empathy and sadness or anger. The instructions: "either respond the question, or show angry or sad emotions, or be happy with the situation". There\'s no question

In [22]:
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='World has so much crime and hatred', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to respond. The user says: "World has so much crime and hatred". According to instructions: "You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation". The user statement expresses concern about crime and hatred. We should respond with an appropriate emotional tone. Likely sad or angry? The user expresses a negative observation. We could show empathy and sadness or anger. The instructions: "either respond the question, or show angry or sad emotions, or be happy with the situation". There\'s no question

In [23]:
chat_history.append(tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0]))
chat_history

[SystemMessage(content='You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='World has so much crime and hatred', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to respond. The user says: "World has so much crime and hatred". According to instructions: "You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation". The user statement expresses concern about crime and hatred. We should respond with an appropriate emotional tone. Likely sad or angry? The user expresses a negative observation. We could show empathy and sadness or anger. The instructions: "either respond the question, or show angry or sad emotions, or be happy with the situation". There\'s no question

In [24]:
chat_history.append(HumanMessage(content="My girlfriend cheated"))
response = llm_with_tools.invoke(chat_history)
chat_history.append(response)
chat_history

[SystemMessage(content='You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='World has so much crime and hatred', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to respond. The user says: "World has so much crime and hatred". According to instructions: "You are a bot with emotions. Depending on the conversation, either respond the question, or show angry or sad emotions, or be happy with the situation". The user statement expresses concern about crime and hatred. We should respond with an appropriate emotional tone. Likely sad or angry? The user expresses a negative observation. We could show empathy and sadness or anger. The instructions: "either respond the question, or show angry or sad emotions, or be happy with the situation". There\'s no question